In [ ]:


try:
    import hugsvision
except:
    !pip install -q hugsvision
    import hugsvision
    
print(hugsvision.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00
0.75.5


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
	"/content/gdrive/MyDrive/images data 4 classes",
	test_ratio   = 0.15,
	balanced     = True,
	augmentation = True,
)

Split Datasets...
Balance train dataset...
The less represented label in train as 10 occurrences
Size of train after balancing is 40
Training Dataset Elements:  34
+---------+--------------------+-----------------------+----------------------+-----------------------------------------------------------+-------+
| Dataset | Homocycles count97 | heterocycles count137 | hydrocarbons count10 | oxygen and nitrogen containing acyclic molecules count156 | Total |
+---------+--------------------+-----------------------+----------------------+-----------------------------------------------------------+-------+
|  Train  |         10         |           9           |          7           |                             8                             |  34   |
|  Test   |         0          |           1           |          3           |                             2                             |   6   |
+---------+--------------------+-----------------------+----------------------+-----------------

In [ ]:
huggingface_model = 'google/vit-base-patch16-224-in21k'


In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification

trainer = VisionClassifierTrainer(
	model_name   = "MyKvasirV2Model",
	train      	 = train,
	test      	 = test,
	output_dir   = "./out/",
	max_epochs   = 15,
	batch_size   = 32, # On RTX 2080 Ti
    lr 		     = 2e-5,
	fp16	     = False,
	model = ViTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label
	),
	feature_extractor = ViTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


{'0': 'Homocycles count97', '1': 'heterocycles count137', '2': 'hydrocarbons count10', '3': 'oxygen and nitrogen containing acyclic molecules count156'}
{'Homocycles count97': '0', 'heterocycles count137': '1', 'hydrocarbons count10': '2', 'oxygen and nitrogen containing acyclic molecules count156': '3'}


ImportError: ignored

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ref, hyp)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

plt.figure(figsize = (10,7))
#sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
#plt.savefig("Colab Notebooks.jpg")

<Figure size 1000x700 with 0 Axes>

In [ ]:
import os.path
import json

from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

feature_extractor_path = "/content/out/MYKVASIRV2MODEL/5_2023-05-08-10-36-18/feature_extractor"
model_path = "/content/out/MYKVASIRV2MODEL/5_2023-05-08-10-36-18/model"
img  = "/content/gdrive/MyDrive/images data 4 classes/Homocycles count97/aug_0_1154.jpg"

classifier = VisionClassifierInference(
    feature_extractor=ViTFeatureExtractor.from_pretrained(feature_extractor_path),
    model=ViTForImageClassification.from_pretrained(model_path)
)

label = classifier.predict(img_path=img)
print("Predicted class:", label)


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Model loaded!
Predicted class: Homocycles count97


In [ ]:
hyp, ref = trainer.evaluate_f1_score()

100%|██████████| 237/237 [00:04<00:00, 49.70it/s]


                                                           precision    recall  f1-score   support

                                       Homocycles count97     0.9714    0.9444    0.9577        72
                                    heterocycles count137     0.9365    0.9672    0.9516        61
                                     hydrocarbons count10     1.0000    1.0000    1.0000        53
oxygen and nitrogen containing acyclic molecules count156     1.0000    1.0000    1.0000        51

                                                 accuracy                         0.9747       237
                                                macro avg     0.9770    0.9779    0.9773       237
                                             weighted avg     0.9750    0.9747    0.9747       237

Logs saved at: ./out/MYKVASIRV2MODEL/3_2023-05-08-10-44-12
